In [ ]:
import sagas.crmsfa.odoo_facade
import imp
imp.reload(sagas.crmsfa.odoo_facade)

In [1]:
from sagas.tool.app_settings import setup_logger, setup_jupyter
setup_logger('crmsfa.log')
setup_jupyter()

In [2]:
from sagas.crmsfa.odoo_facade import odoo, login
login()

if 'sale.order' in odoo.env:
    Order = odoo.env['sale.order']
    order_ids = Order.search([])
    print(order_ids)
else:
    print("no sale.order model")

2019-01-14 00:03:36,305 - sagas.crmsfa.odoo_facade - INFO - ['sagas_odoo']


no sale.order model


In [ ]:
user = odoo.env.user
print(user.name)            # name of the user connected
print(user.company_id.name) # the name of its company

In [ ]:
Country = odoo.env['res.country']
de_id = Country.search([('code', '=', 'DE')])[0]
de = Country.browse(de_id)
print(de)
print(de.name)

In [19]:
import odoorpc.fields as fields
from odoorpc.env import FIELDS_RESERVED
from tabulate import tabulate
import wcwidth

def desc_model(model):
    table_header = ['name','type', 'string']
    table_data = []

    attrs = {
                '_name': model,
                '_columns': {},
            }
    fields_get = odoo.execute(model, 'fields_get')
    for field_name, field_data in fields_get.items():
        if field_name not in FIELDS_RESERVED:
            Field = fields.generate_field(field_name, field_data)
            attrs['_columns'][field_name] = Field
            attrs[field_name] = Field
            # print(field_name, "%", field_data['type'])
            # print(field_data.keys())
            # ['searchable', 'sortable', 'depends', 'store', 
            # 'manual', 'type', 'company_dependent', 
            # 'change_default', 'readonly', 'string', 'required']
            table_data.append((field_name, field_data['type'], 
                               field_data['string']))
            # addi: type(Field)

    # print(tabulate(table_data, headers=table_header, tablefmt='grid'))
    # print(tabulate(table_data, headers=table_header, tablefmt='fancy_grid'))
    print(tabulate(table_data, headers=table_header, tablefmt='psql'))

model="res.country"
desc_model(model)

+-------------------+-----------+------------------------+
| name              | type      | string                 |
|-------------------+-----------+------------------------|
| __last_update     | datetime  | Last Modified on       |
| code              | char      | Country Code           |
| address_view_id   | many2one  | Input View             |
| address_format    | text      | Layout in Reports      |
| name_position     | selection | Customer Name Position |
| state_ids         | one2many  | States                 |
| currency_id       | many2one  | Currency               |
| display_name      | char      | Display Name           |
| name              | char      | Country Name           |
| create_date       | datetime  | Created on             |
| country_group_ids | many2many | Country Groups         |
| image             | binary    | Image                  |
| write_date        | datetime  | Last Updated on        |
| write_uid         | many2one  | Last Updated by       

In [21]:
desc_model('mail.thread')

+----------------------------+-----------+-------------------------+
| name                       | type      | string                  |
|----------------------------+-----------+-------------------------|
| message_partner_ids        | many2many | Followers (Partners)    |
| message_follower_ids       | one2many  | Followers               |
| message_ids                | one2many  | Messages                |
| message_unread             | boolean   | Unread Messages         |
| website_message_ids        | one2many  | Website Messages        |
| message_needaction_counter | integer   | Number of Actions       |
| message_needaction         | boolean   | Action Needed           |
| display_name               | char      | Display Name            |
| message_is_follower        | boolean   | Is Follower             |
| message_last_post          | datetime  | Last Message Date       |
| __last_update              | datetime  | Last Modified on        |
| message_unread_counter     | int

In [22]:
desc_model('crm.lead')

+----------------------------+-----------+-------------------------+
| name                       | type      | string                  |
|----------------------------+-----------+-------------------------|
| message_partner_ids        | many2many | Followers (Partners)    |
| user_login                 | char      | User Login              |
| email_from                 | char      | Email                   |
| message_unread             | boolean   | Unread Messages         |
| medium_id                  | many2one  | Medium                  |
| partner_id                 | many2one  | Customer                |
| title                      | many2one  | Title                   |
| source_id                  | many2one  | Source                  |
| display_name               | char      | Display Name            |
| referred                   | char      | Referred By             |
| name                       | char      | Opportunity             |
| user_id                    | man

In [25]:
model="res.partner"
model_exists = odoo.execute('ir.model', 'search',
                                          [('model', '=', model)])
print(bool(model_exists))

True


In [6]:
from tabulate import tabulate
import wcwidth
from sagas.crmsfa.odoo_facade import odoo, login

def model_info():
    login()

    # models = odoo.execute('ir.model', 'browse', [])
    # print(models)

    # https://stackoverflow.com/questions/2872512/python-truncate-a-long-string
    def abbrev(data, l=15):
        info = (data[:l] + '..') if len(data) > l else data
        return info

    table_header = ['model','name', 'count']
    table_data = []

    Order = odoo.env['ir.model']
    order_ids = Order.search([])
    print(len(order_ids))
    models=Order.browse(order_ids)
    # print(models[0])
    # odoo/addons/base/ir/ir_model.py
    for order in models:
        # print(order.model, order.name, order.count)    
        table_data.append((abbrev(order.model), 
                           abbrev(order.name), order.count))

    print(tabulate(table_data, headers=table_header, tablefmt='psql'))    

model_info()

199
+-------------------+-------------------------------+---------+
| model             | name                          |   count |
|-------------------+-------------------------------+---------|
| product.product   | 产品                          |      40 |
| product.templat.. | 产品模板                      |      36 |
| project.task      | 任务                          |      39 |
| account.analyti.. | 分析账户                      |      23 |
| calendar.event    | 活动                          |       7 |
| mail.test.simpl.. | 测试简单的聊天记录            |       0 |
| mail.test         | 测试邮件模型                  |       0 |
| mail.thread       | 电邮线程                      |       0 |
| crm.lead          | 线索/商机                     |      27 |
| res.partner       | 联系人                        |      43 |
| mail.channel      | 讨论频道                      |       5 |
| crm.team          | 销售渠道                      |       3 |
| project.project   | 项目                          |       6 |
| base              | 

In [4]:
tabulate.WIDE_CHARS_MODE=True